In [1]:
from plutus.data import generate_random_tf_example, generate_random_grpc_query, generate_random_pandas
from plutus.features import default_model_features, default_model_targets, get_model_features
import grpc
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
import tensorflow as tf

In [9]:
x = generate_random_tf_example(default_model_features, default_model_targets)
print(x)

features {
  feature {
    key: "AdFormat"
    value {
      int64_list {
        value: 69
      }
    }
  }
  feature {
    key: "AdsTxtSellerType"
    value {
      int64_list {
        value: 5
      }
    }
  }
  feature {
    key: "AuctionBidPrice"
    value {
      float_list {
        value: 0.1402319073677063
      }
    }
  }
  feature {
    key: "Browser"
    value {
      int64_list {
        value: 17
      }
    }
  }
  feature {
    key: "City"
    value {
      int64_list {
        value: 51562
      }
    }
  }
  feature {
    key: "Country"
    value {
      int64_list {
        value: 208
      }
    }
  }
  feature {
    key: "DealId"
    value {
      int64_list {
        value: 3161
      }
    }
  }
  feature {
    key: "DeviceMake"
    value {
      int64_list {
        value: 517
      }
    }
  }
  feature {
    key: "DeviceModel"
    value {
      int64_list {
        value: 679
      }
    }
  }
  feature {
    key: "DeviceType"
    value {
      int64_list 

Start docker
Start Docker

```sh
docker run -it --cpus=1 --rm --entrypoint=/bin/bash -p 8500:8500 -p 8501:8501 -v /local/models/:/models -t tensorflow/serving:latest
```

To allow for multiple versions of the same model you need (see [docs](https://www.tensorflow.org/tfx/serving/serving_config))
* use a config file
* TF Serving will only load the highest version by default
    * Need to pass this arg to allow multiple versions
    * --allow_version_labels_for_unavailable_models


```sh
tensorflow_model_server --port=8500 --rest_api_port=0 --prefer_tflite_model --model_config_file=/models/model_config.conf --allow_version_labels_for_unavailable_models
```

Important:
* expected to have version in the path

```json
model_config_list: {
    config: {
        name: "MODEL_NAME",
        base_path: "/models/MODEL_DIR",
        model_platform: "tensorflow",
        model_version_policy: {
            specific {
                versions: 1
                versions: 2
            }
        }
        version_labels {
          key: 'prod'
          value: 1
        }
        version_labels {
          key: 'test'
          value: 2
        }
    }
}
```

In [2]:
def send_grpc_request(stub, req, timeout=10):
    return stub.Predict(req, timeout)

channel = grpc.insecure_channel('0.0.0.0:8500')



In [3]:
# Reduced set of features

reduced_model_features = get_model_features(["DealId", "AdFormat", "ImpressionPlacementId"])
reduced_model_features


[Feature(name='SupplyVendor', sparse=True, type=tf.int32, cardinality=102, default_value=0, enabled=True, embedding_dim=16, qr_embed=False, qr_collisions=4),
 Feature(name='SupplyVendorPublisherId', sparse=True, type=tf.int32, cardinality=15002, default_value=0, enabled=True, embedding_dim=16, qr_embed=False, qr_collisions=4),
 Feature(name='SupplyVendorSiteId', sparse=True, type=tf.int32, cardinality=102, default_value=0, enabled=True, embedding_dim=16, qr_embed=False, qr_collisions=4),
 Feature(name='Site', sparse=True, type=tf.int32, cardinality=350002, default_value=0, enabled=True, embedding_dim=16, qr_embed=False, qr_collisions=4),
 Feature(name='Country', sparse=True, type=tf.int32, cardinality=252, default_value=0, enabled=True, embedding_dim=16, qr_embed=False, qr_collisions=4),
 Feature(name='Region', sparse=True, type=tf.int32, cardinality=4002, default_value=0, enabled=True, embedding_dim=16, qr_embed=False, qr_collisions=4),
 Feature(name='Metro', sparse=True, type=tf.int3

In [4]:
# Check that the model_spec is sent for all requests
# It is.


stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
req = generate_random_grpc_query(reduced_model_features, "test")
req
resp = send_grpc_request(stub, req)
print(resp)


outputs {
  key: "params"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 2
      }
    }
    float_val: -2.4589922428131104
    float_val: 1.0099029541015625
  }
}
model_spec {
  name: "test"
  version {
    value: 1
  }
  signature_name: "serving_default"
}



In [ ]:
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
resp = send_grpc_request(stub, generate_random_grpc_query(default_model_features, "test", "prod"))
print(resp)

In [ ]:
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)
resp = send_grpc_request(stub, generate_random_grpc_query(default_model_features, "test", "test"))
print(resp)



In [2]:
x, y = generate_random_pandas(default_model_features, default_model_targets, 100)
x

,SupplyVendor,DealId,SupplyVendorPublisherId,SupplyVendorSiteId,Site,AdFormat,ImpressionPlacementId,Country,Region,Metro,...,OperatingSystemFamily,Browser,sin_hour_day,cos_hour_day,sin_minute_hour,cos_minute_hour,sin_hour_week,cos_hour_week,latitude,longitude
0,81,4558,3033,74,235099,15,46,141,2620,55,...,9,2,0.613107,0.286050,0.464445,0.959238,0.728575,0.508642,0.998087,0.447324
1,63,518,6672,82,123801,2,27,152,841,268,...,7,19,0.205387,0.540049,0.675869,0.945711,0.825966,0.875394,0.670808,0.059899
2,93,856,5381,82,47547,95,14,106,3856,218,...,1,18,0.835652,0.495238,0.365037,0.571576,0.878145,0.341853,0.643675,0.419660
3,11,2149,7810,82,268013,12,95,59,1734,248,...,6,3,0.311508,0.058138,0.906627,0.544105,0.851513,0.328716,0.346650,0.297661
4,40,254,3954,81,328768,34,9,233,1894,64,...,8,16,0.812075,0.370757,0.963082,0.708302,0.523519,0.757354,0.958157,0.506476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,48,4670,7766,12,167070,87,9,131,1463,66,...,1,12,0.199678,0.644879,0.513258,0.685122,0.055030,0.014696,0.321422,0.652607
96,42,972,9546,36,308062,93,58,55,763,298,...,6,16,0.604036,0.665467,0.396411,0.771576,0.072035,0.482076,0.258869,0.067702
97,22,2762,13011,82,214451,21,24,206,1413,128,...,8,14,0.898811,0.491686,0.480014,0.208571,0.326156,0.001399,0.385552,0.404944
98,82,1566,8844,48,324233,31,78,152,966,256,...,0,5,0.629389,0.648813,0.021047,0.416743,0.147122,0.050107,0.108664,0.181052


In [3]:
ds = tf.data.Dataset.from_tensor_slices((x, y))

In [4]:
for a,b in ds.batch(1).take(3):
    print(a)
    print(b)

tf.Tensor(
[[8.10000000e+01 4.55800000e+03 3.03300000e+03 7.40000000e+01
  2.35099000e+05 1.50000000e+01 4.60000000e+01 1.41000000e+02
  2.62000000e+03 5.50000000e+01 2.08200000e+04 4.61010000e+04
  7.19000000e+02 2.58500000e+03 2.64000000e+02 1.00000000e+00
  3.90000000e+01 4.00000000e+00 0.00000000e+00 4.00000000e+00
  9.00000000e+00 2.00000000e+00 6.13106602e-01 2.86049924e-01
  4.64444985e-01 9.59237998e-01 7.28574848e-01 5.08641660e-01
  9.98086825e-01 4.47323799e-01]], shape=(1, 30), dtype=float64)
tf.Tensor([[0.11560559 0.26812042 0.17970432 0.42399712 0.1107953 ]], shape=(1, 5), dtype=float64)
tf.Tensor(
[[6.30000000e+01 5.18000000e+02 6.67200000e+03 8.20000000e+01
  1.23801000e+05 2.00000000e+00 2.70000000e+01 1.52000000e+02
  8.41000000e+02 2.68000000e+02 6.80800000e+03 3.26790000e+04
  4.10000000e+01 2.67000000e+02 2.07000000e+02 4.00000000e+00
  1.54000000e+02 4.00000000e+00 1.00000000e+00 6.00000000e+00
  7.00000000e+00 1.90000000e+01 2.05386947e-01 5.40048847e-01
  6.7586